In [4]:
import numpy as np
from helpers import *

from implementations import *
import zipfile
import os
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
zip_path = 'dataset_to_release.zip'
save_path = 'project1/'
file = zipfile.ZipFile(zip_path)
file.extractall(save_path)
file.close()
data_path = 'project1/dataset_to_release'

In [7]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data(data_path, sub_sample=False)

In [4]:
check_array_shapes(x_train, x_test, y_train, train_ids, test_ids)

(328135, 321)
(109379, 321)
(328135,)
(328135,)
(109379,)


In [5]:
tX = x_train
y = y_train
ids = train_ids


According to the information on website, we delete the first 10 columns contains the information unrelated to the final prediction.

In [6]:
tX = tX[:, 10:]

Print the ratio of missing values for each feature

In [7]:
# analyze_missing_values(tX, y)

As some data means "not known", "refuse to answer", we replace those number with NaN. For the data means 0 in value, we replace them with 0.001

In [8]:
values_to_replace_with_nan = [77, 99, 7, 9, 777, 999, 777777, 999999, 98,7777,9999,89]
values_to_replace_with_0_001 = [888,88, 8]

replace_values(tX, values_to_replace_with_nan, values_to_replace_with_0_001)

There is some NaN values in the array, we first delete the feartures. Then we do PCA analysis on the remained features and use k-means cluster to cluster and label each sample.

In [9]:
tX_NoNaN = remove_columns_with_nan(tX)
print(tX_NoNaN.shape)

(328135, 17)


As the dimension of the tX data is high, we first reduce the dimension and then use K-means clustering

In [10]:
n_components = 15  # Number of principal components to retain
X_reduced_feature = perform_pca(tX_NoNaN, n_components)
X_reduced_feature.shape

(328135, 15)

In [11]:
n_clusters = 10  # Number of clusters
cluster_assignment = k_means_clustering(X_reduced_feature, n_clusters)
print(cluster_assignment.shape)

(328135,)


Now we already know which cluster each sample belongs to, we delete the featrues that contains too much NaN values. Here we delete the feature clolumns with more than 80% NaN value.

In [12]:
missing_value_threshold = 0.2
remained_columns = clean_dataset(tX, missing_value_threshold)
cleaned_tX = tX[:, remained_columns]
print(cleaned_tX.shape)

(328135, 129)


For the remained array, the number of NaN for each feture is not too much. Therefore, we replace the NaN value with corresponding cluster median.

In [13]:
tX_filled = fill_missing_with_median(cleaned_tX, cluster_assignment)
train_medians = np.nanmedian(tX_filled, axis=0)

Normalize X

In [15]:
normalized_x = normalize(tX_filled)

In [16]:
w_value = 1e-1
tx_expanded, initial_w = setup_features_and_weights(normalized_x, w_value)

We convert -1 to 0 for y, as we use logistic regression for classification

In [17]:
y_new = np.where(y == -1, 0, y)

In [20]:
start_time = time.time()
# Set the hyperparameters
gammas = np.logspace(-0.75, -0.67, 3)
lambdas = np.logspace(-1, 0, 3)# Regularization parameter
k_fold = 10

# Call logistic_regression_gd function for logistic regression
gamma, lambda_= best_selection(y_new, tx_expanded, initial_w, k_fold, lambdas, gammas)

end_time = time.time()

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done


In [21]:
print(gamma, lambda_)

0.1778279410038923 0.1


In [22]:
final_w, final_loss = reg_logistic_regression(y=y_new, tx=tx_expanded, initial_w=initial_w, gamma=gamma, lambda_=lambda_)
print("Final Loss:", final_loss)

Final Loss: 0.2838681457547004


In [36]:
print('run time:', end_time-start_time)

run time: 101.16616606712341


In [27]:
tx_test = preprocess_data(x_test,values_to_replace_with_nan, values_to_replace_with_0_001, n_components, n_clusters, missing_value_threshold, remained_columns, train_medians)

/Users/don_giraffe/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)


In [31]:
raw_predictions = np.dot(tx_test, final_w)
predicted_probabilities = 1 / (1 + np.exp(-raw_predictions))

[-0.98700167 -1.25850318 -0.81194227 ... -0.80006917 -0.53972661
 -0.75015857]
0.6363503675066515


Make predictions on the test set using the final weight obtained from training.

In [32]:
test_predictions = predict(tx_test, final_w)
y_pred = np.where(test_predictions == 0, -1, test_predictions)

In [35]:
create_csv_submission(test_ids, y_pred, name='y_test.csv')